In [ ]:
import sys

sys.path.append("../..")
from experiments.fourier_miniworld_gridsearch.common import (
    generate_env,
    create_agent_for_test,
)
from experiments.fourier_miniworld_gridsearch.loop_tests import loop_path
from fourier_vectorhash import path_test, AdditiveCombine, MultiplicativeCombine
from fourier_scaffold import (
    GuassianFourierSmoothingMatrix,
    HadamardShiftMatrixRat,
    HadamardShiftMatrix,
    ContractionSharpening,
)
import torch
import pickle

results_dir = "july_30"


In [ ]:
combination_31 = (
    800,
    "no_cnn",
    MultiplicativeCombine(),
    1,
    GuassianFourierSmoothingMatrix([10] * 3, [0.5] * 3),
    HadamardShiftMatrix(),
    ContractionSharpening(k=1),
)
combination_39 = (
    800,
    "no_cnn_extra_downscaling",
    AdditiveCombine(0.1),
    1,
    GuassianFourierSmoothingMatrix([10] * 3, [0.3] * 3),
    HadamardShiftMatrix(),
    ContractionSharpening(k=1),
)
combination_47 = (
    800,
    "no_cnn_extra_downscaling",
    AdditiveCombine(0.1),
    1,
    GuassianFourierSmoothingMatrix([10] * 3, [0.1] * 3),
    HadamardShiftMatrix(),
    ContractionSharpening(k=1),
)

In [ ]:
env = generate_env(with_red_box=True, with_blue_box=True)
combination, combination_name = combination_31, "test_31"
# combination, combination_name = combination_31, "test_39"
# combination, combination_name = combination_31, "test_47"
history = path_test(
    agent=create_agent_for_test(env, *combination),
    path=torch.tensor(loop_path),
    reshape_img_size=(15, 20),
)

In [ ]:
import os
os.makedirs(results_dir, exist_ok=True)
with open(f"{results_dir}/{combination_name}.pkl", "wb") as f:
    pickle.dump(history[0], f)

In [ ]:
from experiments.fourier_miniworld_gridsearch.common import (
  analyze_history_errors
)
fig = analyze_history_errors(history[0], 240)
fig.savefig(f"{results_dir}/{combination_name}.png")

In [ ]:
anim = history[0].make_image_video()
anim.save(
    f"{results_dir}/{combination_name}",
    progress_callback=lambda frame, total: print(f"{frame+1}/{total}"),
)